# AI-LAB SESSION 1: Uninformed search

In this first session we will work on uninformed search

## Maze environments

The environments used are **SmallMaze** (visible in the figure) and its variations
![SmallMaze](images/maze.png)
The agent starts in cell $(0, 2)$ and has to reach the treasure in $(4, 3)$

## Assignment 1

Your first assignment is to implement the BFS algorithm on **SmallMaze**. In particular, you are required to implement both *tree_search* and *graph_search* versions of BFS that will be called by the generic *bfs*.

The results returned by your *bfs* must be a tuple $(path, stats)$ in the following form:
* *path* - tuple of state identifiers forming a path from the start state to the goal state. ``None`` if no solution is found
* *stats* - tuple of:
     * *time* - time elapsed between the start and the end of the algorithm
     * *expc* - number of nodes explored. A node is considered as explored when removed from the fringe and analyzed
     * *maxnodes* - maximum number of nodes in memory at the same time (fringe + closed)

After the correctness of your implementations have been assessed, you can run the algorithms on other two maze environments: **GrdMaze** and **BlockedMaze**.

Function *build_path* can be used to return a tuple of states from the root node (excluded) to another node by following *parent* links

In [10]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import gym
import envs
from timeit import default_timer as timer
from utils.fringe import FringeNode, QueueFringe
from utils.funcs import build_path

node_1 = FringeNode(1)
node_2 = FringeNode(2, node_1)
node_3 = FringeNode(3, node_2)
build_path(node_3)

(2, 3)

The next two functions have to be implemented

In [13]:
def tree_search(environment, fringe):
    """
    Tree search
    
    Args:
        environment: OpenAI Gym environment
        fringe: instance of Fringe data structure
        
    Returns:
        (path, stats): solution as a path and stats.
        The stats are a tuple of (expc, maxnodes): number of explored states, max nodes in memory
    """
    removed = 0
    node = FringeNode(environment.startstate)
    if node.state == environment.goalstate:
        return build_path(node), (removed, len(fringe))
    fringe.add(node)
    
    while True:
        if fringe.is_empty():
            return None, (removed, len(fringe))
        node = fringe.remove()
        removed = removed + 1
        
        if node.state == environment.goalstate:
            return build_path(node), (removed, len(fringe) + 1)
        
        for action in range(environment.action_space.n):
            child = FringeNode(environment.sample(node.state, action), node)
            fringe.add(child)

In [11]:
def graph_search(environment, fringe):
    """
    Graph search
    
    Args:
        environment: OpenAI Gym environment
        fringe: instance of Fringe data structure
        
    Returns:
        (path, stats): solution as a path and stats.
        The stats are a tuple of (expc, maxnodes): number of explored nodes, max nodes in memory
    """
    removed = 0
    node = FringeNode(environment.startstate)
    if node.state == environment.goalstate:
        return build_path(node), (removed, len(fringe) + len(closed))
    fringe.add(node)
    closed = QueueFringe()
    while True:
        if fringe.is_empty():
            return None, (removed, len(fringe) + len(closed))
        node = fringe.remove()
        removed = removed + 1
        closed.add(node)
        
        for action in range(environment.action_space.n):
            child = FringeNode(environment.sample(node.state, action), node)
            if child.state not in fringe and child.state not in closed:
                if child.state == environment.goalstate:
                    return build_path(child), (removed + 1, len(fringe) + len(closed) + 1)
                fringe.add(child)

In [12]:
def bfs(environment, search_type):
    """
    Breadth-first search
    
    Args:
        environment: OpenAI Gym environment
        search_type: type of search - tree_search or graph_search (function pointer)
        
    Returns:
        (path, stats): solution as a path and stats.
        The stats are a tuple of (time, expc, maxnodes): elapsed time, number of explored nodes, max nodes in memory
    """
    t = timer()
    path, stats = search_type(environment, QueueFringe())
    return path, (timer() - t, stats[0], stats[1])

The following code calls your tree search version of BFS and prints the results

In [14]:
envname = "GrdMaze-v0"  # Other options are GrdMaze-v0 and BlockedMaze-v0

print("\n----------------------------------------------------------------")
print("\tTREE SEARCH")
print("\tEnvironment: ", envname)
print("----------------------------------------------------------------\n")

# Create and render the environment
env = gym.make(envname)

# Create and render the environment
env = gym.make(envname)
env.render()
solution, stats = bfs(env, tree_search)  # Perform BFS
if solution is not None:
    solution = [env.state_to_pos(s) for s in solution]
    
# Print stats and path
print("\n\nExecution time: {0}s\nN° of nodes explored: {1}\nMax n° of nodes in memory: {2}\nSolution: {3}".format(
        round(stats[0], 4), stats[1], stats[2], solution))


----------------------------------------------------------------
	TREE SEARCH
	Environment:  GrdMaze-v0
----------------------------------------------------------------

[['C' 'C' 'C' 'S']
 ['C' 'C' 'W' 'C']
 ['C' 'C' 'C' 'C']
 ['C' 'W' 'W' 'W']
 ['C' 'C' 'C' 'G']]


Execution time: 52.9909s
N° of nodes explored: 365867
Max n° of nodes in memory: 1097599
Solution: [(0, 2), (0, 1), (0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (4, 1), (4, 2), (4, 3)]


Correct results for BFS tree search can be found [here](results/bfs_tree_search_results.txt)

The following code calls your graph search version of BFS and prints the results

In [9]:
envname = "SmallMaze-v0"  # Other options are GrdMaze-v0 and BlockedMaze-v0

print("\n----------------------------------------------------------------")
print("\tGRAPH SEARCH")
print("\tEnvironment: ", envname)
print("----------------------------------------------------------------\n")

# Create and render the environment
env = gym.make(envname)
env.render()
solution, stats = bfs(env, graph_search)  # Perform BFS
if solution is not None:
    solution = [env.state_to_pos(s) for s in solution]
    
# Print stats and path
print("\n\nExecution time: {0}s\nN° of nodes explored: {1}\nMax n° of nodes in memory: {2}\nSolution: {3}".format(
        round(stats[0], 4), stats[1], stats[2], solution))


----------------------------------------------------------------
	GRAPH SEARCH
	Environment:  SmallMaze-v0
----------------------------------------------------------------

[['C' 'C' 'S' 'C']
 ['C' 'C' 'W' 'C']
 ['C' 'C' 'C' 'C']
 ['C' 'W' 'W' 'W']
 ['C' 'C' 'C' 'G']]


Execution time: 0.008s
N° of nodes explored: 16
Max n° of nodes in memory: 16
Solution: [(0, 1), (0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (4, 1), (4, 2), (4, 3)]


Correct results for BFS graph search can be found [here](results/bfs_graph_search_results.txt)

## Assignment 2

Your second assignment is to implement the IDS algorithm on **SmallMaze**. In particular, you are required to implement both *dls_ts* (depth-limited tree search) and *dls_gs* (depth-limited graph search) versions of IDS that will be called by the generic *ids*. The recursions must be implemented in *rdls_ts* (recursive depth-limited tree search) and *rdls_gs* (recursive depth-limited graph search) called by *dls_ts* and *dls_gs* respectively.

Similarly to assignment 1, the results returned by your *ids* must be a tuple $(path, stats)$ in the following form:
* *path* - tuple of state identifiers forming a path from the start state to the goal state. ``None`` if no solution is found
* *stats* - tuple of:
     * *time* - time elapsed between the start and the end of the algorithm
     * *expc* - number of nodes explored. A node is considered as explored when removed from the fringe and analyzed
     * *maxnodes* - maximum number of nodes in memory at the same time (the depth of the recursion stack + closed)

After the correctness of your implementations have been assessed, you can run the algorithms on other two maze environments: **GrdMaze** and **BlockedMaze**.

**FringeNode** has a useful variable that can be set in the constructor and can be used to track the depth of a node in the path (and consequently of the recursion stack of IDS): *pathcost*. If the root node has a *pathcost* of 0, its children will have a *pathcost* increased by 1

In [15]:
start = env.startstate
root = FringeNode(start)  # parent = None and pathcost = 0 as default
child = FringeNode(env.sample(start, 0), root, root.pathcost + 1)  # pathcost is the third argument
print("Root pathcost: {}\tChild pathcost: {}".format(root.pathcost, child.pathcost))

Root pathcost: 0	Child pathcost: 1


Here you can implement the various functions requested

In [17]:
def dls_ts(environment, limit):
    """
    Depth-limited search (tree search)
    
    Args:
        environment: OpenAI Gym environment
        limit: depth limit budget
        
    Returns:
        (path, cutoff, stats): solution as a path, cutoff flag and stats.
        The stats are a tuple of (time, expc, maxnodes): elapsed time, number of explored nodes, max nodes in memory
    """
    t = timer()
    path, cutoff, expc, maxdepth = rdls_ts(environment, FringeNode(environment.startstate), limit)
    return path, cutoff, (timer() - t, expc, maxdepth)

In [16]:
def rdls_ts(environment, node, limit):
    """
    Recursive depth-limited search (tree search)
    
    Args:
        environment: OpenAI Gym environment
        node: node to explore
        limit: depth limit budget
    
    Returns:
        (path, cutoff, expc, maxdepth): path, cutoff flag, number of explored nodes, max nodes in memory
    """
    if node.state == environment.goalstate:
        return build_path(node), False, 1, limit + 1
    
    if limit == 0:
        return None, True,0,0
    
    cutoffOccured = False
    counter = 1
    for action in range(environment.action_space.n):
        child = FringeNode(environment.sample(node.state, action), node)
        counter = counter + 1
        path, cutoff, expc, maxDepth = rdls_ts(environment, child, limit - 1)
        if cutoff == True:
            cutoffOccured = True
        elif path != None:
            return path, cutoff, expc + counter, maxDepth
    
    if cutoffOccured:
        return None, True, expc + counter, 0
    return None, False, expc + counter, 0

In [42]:
 def dls_gs(environment, limit):
    """
    Depth-limited search (graph search)
    
    Args:
        environment: OpenAI Gym environment
        limit: depth limit budget
        
    Returns:
        (path, cutoff, stats): solution as a path, cutoff flag and stats.
        The stats are a tuple of (time, expc, maxnodes): elapsed time, number of explored nodes, max nodes in memory
    """
    t = timer()
    path, cutoff, expc, maxdepth = rdls_gs(environment, FringeNode(environment.startstate), limit, QueueFringe())
    return path, cutoff, (timer() - t, expc, maxdepth)

In [49]:
def rdls_gs(environment, node, limit, closed):
    """
    Recursive depth-limited search (graph search)
    
    Args:
        environment: OpenAI Gym environment
        node: node to explore
        limit: depth limit budget
        closed: completely explored nodes
        
    Returns:
        (path, cutoff, expc, maxdepth): path, cutoff flag, n° of nodes explored, max nodes in memory
    """
    if node.state == environment.goalstate:
        return build_path(node), False, 1, len(closed)
    
    if limit == 0:
        return None, True,0,0
    
    cutoffOccured = False
    counter = 1
    for action in range(environment.action_space.n):
        child = FringeNode(environment.sample(node.state, action), node)
        counter = counter + 1
        if child.state not in closed:
            closed.add(child)
            path, cutoff, expc, maxDepth = rdls_gs(environment, child, limit - 1, closed)
            if cutoff == True:
                cutoffOccured = True
            elif path != None:
                return path, cutoff, expc + counter, len(closed)
    
    if cutoffOccured:
        return None, True, counter + len(closed), len(closed)
    return None, False, counter + len(closed), len(closed)

In [50]:
def ids(environment, search_type):
    """
    Iterative deepening depth-first search
    
    Args:
        environment: OpenAI Gym environment
        search_type: type of search (graph or tree) - dls_gs or dls_ts (function pointer)
    
    Returns:
        (path, stats): solution as a path and stats.
        The stats are a tuple of (time, expc, maxnodes): elapsed time, number of explored nodes, max nodes in memory
    """
    t = timer()
    expc = 0
    for depth in range(0, sys.maxsize):
        path, cutoff, stats = search_type(environment, depth)
        if cutoff != True:
            expc = expc + stats[1]
            return path, (timer() - t, expc, depth + 1)

The following code calls your tree search version of IDS and prints the results

In [51]:
envname = "SmallMaze-v0"  # Other options are GrdMaze-v0 and BlockedMaze-v0

print("\n----------------------------------------------------------------")
print("\tTREE SEARCH")
print("\tEnvironment: ", envname)
print("----------------------------------------------------------------\n")

# Create and render the environment
env = gym.make(envname)
env.render()
solution, stats = ids(env, dls_ts)  # Perform BFS
if solution is not None:
    solution = [env.state_to_pos(s) for s in solution]
    
# Print stats and path
print("\n\nExecution time: {0}s\nN° of nodes explored: {1}\nMax n° of nodes in memory: {2}\nSolution: {3}".format(
        round(stats[0], 4), stats[1], stats[2], solution))


----------------------------------------------------------------
	TREE SEARCH
	Environment:  SmallMaze-v0
----------------------------------------------------------------

[['C' 'C' 'S' 'C']
 ['C' 'C' 'W' 'C']
 ['C' 'C' 'C' 'C']
 ['C' 'W' 'W' 'W']
 ['C' 'C' 'C' 'G']]


Execution time: 4.5638s
N° of nodes explored: 34
Max n° of nodes in memory: 10
Solution: [(0, 1), (0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (4, 1), (4, 2), (4, 3)]


Correct results for IDS tree search can be found [here](results/ids_tree_search_results.txt)

The following code calls your graph search version of IDS and prints the results

In [52]:
envname = "GrdMaze-v0"  # Other options are GrdMaze-v0 and BlockedMaze-v0

print("\n----------------------------------------------------------------")
print("\tGRAPH SEARCH")
print("\tEnvironment: ", envname)
print("----------------------------------------------------------------\n")

# Create and render the environment
env = gym.make(envname)
env.render()
solution, stats = ids(env, dls_gs)  # Perform BFS
if solution is not None:
    solution = [env.state_to_pos(s) for s in solution]
    
# Print stats and path
print("\n\nExecution time: {0}s\nN° of nodes explored: {1}\nMax n° of nodes in memory: {2}\nSolution: {3}".format(
        round(stats[0], 4), stats[1], stats[2], solution))


----------------------------------------------------------------
	GRAPH SEARCH
	Environment:  GrdMaze-v0
----------------------------------------------------------------

[['C' 'C' 'C' 'S']
 ['C' 'C' 'W' 'C']
 ['C' 'C' 'C' 'C']
 ['C' 'W' 'W' 'W']
 ['C' 'C' 'C' 'G']]


Execution time: 0.0161s
N° of nodes explored: 41
Max n° of nodes in memory: 13
Solution: [(0, 2), (0, 1), (0, 0), (1, 0), (1, 1), (2, 1), (2, 0), (3, 0), (4, 0), (4, 1), (4, 2), (4, 3)]


Correct results for IDS graph search can be found [here](results/ids_graph_search_results.txt)

## Discussion

Now that you have correctly implemented both BFS and IDS what can you say about the solutions they compute? Are there significant differences in the stats?